In [8]:
# Imports

import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# Download required NLTK data files
from nltk.corpus import wordnet

# Initialize SpaCy for tokenization and part of speech tagging
import spacy
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /Users/oliviagao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Loading in Data

Data Description from Github:
_______________________

Word Complexity Lexicon (https://github.com/mounicam/lexical_simplification/tree/master/word_complexity_lexicon)
_______________________


lexicon.tsv : Each line consists of word and its complexity scores calculated by aggregating over human ratings. 
              The score belongs to a scale of 1-6, where 1 represents "very simple" and 6 represents "very complex"

lexion_annotations.tsv: Each line consists of a word in the lexicon and its individual ratings from 11 annotators.
                        Each rating again belongs to the scale of 1-6. -1 indicates that the annotator did not rate
                        the word.
                        
NOTE: Both the files are tab delimited.

In [2]:
lexicon_data = pd.read_csv('../WCL_data/lexicon.csv')
lexicon_ann = pd.read_csv('../WCL_data/lexicon_annotations.csv')

## Data Cleaning

In [3]:
# TODO: 
# Clean Data
# split into train and test
# Get dimensions, summary, etc. of data
print("Lexicon data")
print(lexicon_data.shape)
print(lexicon_data.head())

print("Lexicon ann")
print(lexicon_ann.shape)
print(lexicon_ann.head())

Lexicon data
(15180, 2)
            word  rating
0            wet  1.5714
1          cargo  2.8571
2        Arsenal  3.7143
3  Manufacturing  3.8333
4           East  1.2857
Lexicon ann
(15180, 12)
            word  ann_1  ann_2  ann_3  ann_4  ann_5  ann_6  ann_7  ann_8  \
0            wet     -1      1      1     -1      1      1      2      3   
1          cargo     -1     -1      2     -1      4      2      2      4   
2        Arsenal      4     -1     -1      4      5     -1      4      3   
3  Manufacturing     -1      4      4     -1     -1      3      4      4   
4           East     -1      2      1      1      1     -1      1      2   

   ann_9  ann_10  ann_11  
0     -1       2      -1  
1     -1       3       3  
2      3      -1       3  
3     -1       4      -1  
4      1      -1      -1  
